<a href="https://colab.research.google.com/github/EvilLucifer00/Team1_ACM_AI/blob/main/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
df.shape

In [ ]:
df = pd.read_csv('train.csv')
df.shape

In [ ]:

CATEGORY_KEYWORDS = {
    # Food
    "chocolate": ["chocolate", "cocoa", "dark choc", "milk choc"],
    "bread": ["bread", "bun", "loaf", "bagel", "brioche", "croissant"],
    "butter": ["butter", "margarine", "ghee", "spread"],
    "jam": ["jam", "jelly", "marmalade", "preserve"],
    "coffee": ["coffee", "espresso", "latte", "mocha", "cappuccino"],
    "tea": ["tea", "chai", "matcha", "oolong", "herbal"],
    "milk": ["milk", "dairy", "cream", "creamer"],
    "syrup": ["syrup", "honey", "molasses", "maple"],
    "snack": ["snack", "chips", "crackers", "popcorn", "nuts", "pretzel"],
    "spice": ["spice", "seasoning", "masala", "herb"],
    "sauce": ["sauce", "ketchup", "mustard", "mayonnaise", "dressing", "salsa"],
    "pasta": ["pasta", "noodle", "spaghetti", "macaroni", "lasagna"],
    "rice": ["rice", "grain", "basmati", "brown rice"],
    "biscuit": ["biscuit", "cookie", "cracker", "shortbread"],
    "juice": ["juice", "nectar", "squash"],
    "oil": ["oil", "olive", "sunflower", "canola", "vegetable"],
    "cereal": ["cereal", "granola", "oats", "muesli"],
    "candy": ["candy", "sweet", "toffee", "gum", "mint"],
    "protein": ["protein", "whey", "supplement", "shake", "powder"],
    "vinegar": ["vinegar", "acetic"],
    "pickle": ["pickle", "relish", "gherkin"],
    "drink": ["drink", "beverage", "soda", "cola", "sparkling", "energy drink", "juice"],
    "baking": ["flour", "yeast", "baking powder", "cornstarch", "icing"],
    "cheese": ["cheese", "mozzarella", "cheddar", "parmesan"],
    "frozen": ["frozen", "ice cream", "frozen meal"],
    "soup": ["soup", "broth", "stock"],
    "egg": ["egg", "eggs"],
    "meat": ["meat", "chicken", "mutton", "pork", "beef", "fish"],
    "vegetable": ["vegetable", "veggie", "spinach", "tomato", "onion", "potato"],
    "fruit": ["fruit", "apple", "banana", "mango", "berry", "orange", "grape"],

    # Non-food
    "detergent": ["detergent", "washing powder", "laundry", "fabric softener"],
    "cleaner": ["cleaner", "disinfectant", "bleach", "toilet", "floor", "bathroom"],
    "dishwashing": ["dishwash", "dish soap", "utensil cleaner"],
    "soap": ["soap", "bar soap", "handwash"],
    "shampoo": ["shampoo", "conditioner", "hair wash"],
    "toothpaste": ["toothpaste", "tooth powder", "mouthwash"],
    "skincare": ["lotion", "cream", "moisturizer", "serum", "face wash", "cleanser"],
    "cosmetic": ["makeup", "lipstick", "foundation", "mascara", "eyeliner", "nail polish"],
    "deodorant": ["deodorant", "perfume", "body spray", "fragrance"],
    "sanitary": ["sanitary", "napkin", "pad", "tampon"],
    "baby_care": ["baby", "infant", "diaper", "wipes", "baby food"],
    "pet_food": ["dog food", "cat food", "pet treat", "bird feed"],
    "stationery": ["pen", "pencil", "notebook", "eraser", "marker", "paper"],
    "home_supplies": ["tissue", "napkin", "foil", "wrap", "bag", "cup", "plate"],
    "electronics": ["battery", "bulb", "charger", "adapter", "cable"],
}

SPELLING_CORRECTIONS = {
    # Weight/volume
    "ouunce": "ounce",
    "ouu?nce": "ounce",
    "ounc": "ounce",
    "grm": "gram",
    "graam": "gram",
    "gramme": "gram",
    "grams": "gram",
    "kilo": "kg",
    "kgr": "kg",
    "mililitre": "ml",
    "millilitre": "ml",
    "milliliter": "ml",
    "milileter": "ml",
    "milil": "ml",
    "liter": "l",
    "litre": "l",
    "litr": "l",
    "lt": "l",
    "ltr": "l",
    # Quantity
    "pcs": "pieces",
    "pce": "pieces",
    "peices": "pieces",
    "peece": "pieces",
    "cnt": "count",
    "pk": "pack",
    # Colour
    "colr": "colour",
    "clr": "colour",
    "clour": "colour",
    # Misc
    "flavor": "flavour",
    "flavr": "flavour",
    "favr": "flavour",
}

def normalize_text(text):
    """Lowercase and fix common spelling issues, including fl oz variants."""
    text = str(text).lower()

    # Normalize fluid ounce variants before other corrections
    text = re.sub(r"fl[\.\s]*oz|fluid[\s]*ounce[s]?", "fl_oz", text)

    for wrong, correct in SPELLING_CORRECTIONS.items():
        text = re.sub(rf"\b{wrong}\b", correct, text)
    return text


def detect_category(text):
    """Detect product category based on keywords."""
    for category, keywords in CATEGORY_KEYWORDS.items():
        for kw in keywords:
            if re.search(rf"\b{kw}\b", text):
                return category
    return "other"

def extract_product_info(text):
    """Extract structured info (weight, volume, brand, category, etc.)."""
    text = normalize_text(text)
    info = {}

    # Item Name
    match = re.search(r"item\s*name\s*:\s*(.*)", text)
    info["item_name"] = match.group(1).strip() if match else text.strip()

    # Weight
    match = re.search(r"(\d+(?:\.\d+)?)\s*(g|gram|kg|oz|ounce|lb)", text)
    info["weight_value"] = match.group(1) if match else np.nan
    info["weight_unit"] = match.group(2) if match else "unknown"

    # Volume
    match = re.search(r"(\d+(?:\.\d+)?)\s*(ml|l|fl_oz)", text)
    info["volume_value"] = match.group(1) if match else np.nan
    info["volume_unit"] = match.group(2) if match else "unknown"

    # Imported from
    match = re.search(r"imported\s+from\s+([a-z ]+)", text)
    info["imported_from"] = match.group(1).strip() if match else np.nan

    # Colour
    match = re.search(r"colour\s*:\s*([a-z ]+)", text)
    info["colour"] = match.group(1).strip() if match else np.nan

    # Pack and quantity parsing

    pack_match = re.search(r"(\d+)\s*[- ]*(?:pack|packs?)", text)
    combo_match = re.search(r"(\d+)\s*[x×]\s*(\d+)\s*(g|gram|kg|ml|l|fl_oz|oz|ounce)?", text)
    count_match = re.search(r"(\d+)\s*(?:pieces?|count|pcs|bottles|units|bars|sticks|pouches|bags)", text)

    if combo_match:
        # "12 x 30g" → 12 packs, 30 per pack
        info["pack_count"] = int(combo_match.group(1))
        info["quantity_per_pack"] = int(combo_match.group(2))
    elif pack_match and count_match:
        # "3 pack of 6 bars" → 3 packs, 6 per pack
        info["pack_count"] = int(pack_match.group(1))
        info["quantity_per_pack"] = int(count_match.group(1))
    elif pack_match:
        # "3-pack" → 3 packs of 1
        info["pack_count"] = int(pack_match.group(1))
        info["quantity_per_pack"] = 1
    elif count_match:
        # "6 count" → 1 pack of 6
        info["pack_count"] = 1
        info["quantity_per_pack"] = int(count_match.group(1))
    else:
        # Default → single item
        info["pack_count"] = 1
        info["quantity_per_pack"] = 1


    # Flavour
    match = re.search(r"flavour\s*:\s*([a-z ]+)", text)
    info["flavour"] = match.group(1).strip() if match else np.nan

    # Brand heuristic (first words before descriptors)
    brand_guess = re.match(r"([a-z]+(?:\s+[a-z]+)?)", info["item_name"])
    info["brand_name"] = brand_guess.group(1) if brand_guess else np.nan

    # Category
    info["category"] = detect_category(text)

    return info


extracted = df["catalog_content"].apply(extract_product_info).apply(pd.Series)


result_full = pd.concat([df[["sample_id", "price"]], extracted], axis=1)
print(result_full.head(10))


result_full.to_csv("extracted_features_full.csv", index=False)


In [ ]:
result_full["price"].skew()

In [ ]:
def standardize_units(row):
    weight_val = row.get("weight_value", np.nan)
    weight_unit = str(row.get("weight_unit", "")).lower()
    volume_val = row.get("volume_value", np.nan)
    volume_unit = str(row.get("volume_unit", "")).lower()

    weight_g = np.nan
    volume_ml = np.nan

    #Weight
    if pd.notna(weight_val):
        try:
            value = float(weight_val)
            if weight_unit in ["g", "gram", "grams"]:
                weight_g = value
            elif weight_unit in ["kg", "kilogram"]:
                weight_g = value * 1000
            elif weight_unit in ["oz", "ounce"]:
                weight_g = value * 28.3495
            elif weight_unit in ["lb", "pound"]:
                weight_g = value * 453.592
        except ValueError:
            pass

    # --- Volume ---
    if pd.notna(volume_val):
        try:
            value = float(volume_val)
            if volume_unit in ["ml", "millilitre", "milliliter"]:
                volume_ml = value
            elif volume_unit in ["l", "litre", "liter"]:
                volume_ml = value * 1000
            elif volume_unit in ["fl_oz", "fl oz", "fluid ounce"]:
                volume_ml = value * 29.5735
        except ValueError:
            pass

    return pd.Series({"weight_g": weight_g, "volume_ml": volume_ml})

result_full[["weight_g", "volume_ml"]] = result_full.apply(standardize_units, axis=1)



def compute_cost_features(row):
    cost_per_weight = np.nan
    cost_per_volume = np.nan
    total_weight = np.nan
    total_volume = np.nan

    #Total weight and volume considering pack/quantity
    if pd.notna(row["weight_g"]):
        total_weight = row["weight_g"] * row["pack_count"] * row["quantity_per_pack"]
        cost_per_weight = row["price"] / total_weight if total_weight > 0 else np.nan

    if pd.notna(row["volume_ml"]):
        total_volume = row["volume_ml"] * row["pack_count"] * row["quantity_per_pack"]
        cost_per_volume = row["price"] / total_volume if total_volume > 0 else np.nan

    return pd.Series({
        "total_weight_g": total_weight,
        "total_volume_ml": total_volume,
        "cost_per_weight": cost_per_weight,
        "cost_per_volume": cost_per_volume
    })

result_full[["total_weight_g", "total_volume_ml", "cost_per_weight", "cost_per_volume"]] = result_full.apply(compute_cost_features, axis=1)


In [ ]:
result_full['total_qty'] = (
    result_full['pack_count'].fillna(1).astype(float).replace(0, 1) *
    result_full['quantity_per_pack'].fillna(1).astype(float).replace(0, 1)
)
result_full['unit_price'] = result_full['price'].astype(float) / result_full['total_qty']

In [ ]:
result_full.tail()

In [ ]:
result_full["unit_price"].skew()

In [ ]:
result_full['unit_price'].isna().sum()


In [ ]:
result_full['log_unit_price'] = np.log1p(result_full['unit_price'])
result_full["log_unit_price"].skew()

In [ ]:
result_full.to_csv("cleaned_train_data.csv", index=False)

In [ ]:
dff=pd.read_csv('test.csv')


In [ ]:
def total(row):

    total_weight = np.nan
    total_volume = np.nan

    # Total weight and volume considering pack and quantity
    if pd.notna(row["weight_g"]):
        total_weight = row["weight_g"] * row["pack_count"] * row["quantity_per_pack"]


    if pd.notna(row["volume_ml"]):
        total_volume = row["volume_ml"] * row["pack_count"] * row["quantity_per_pack"]


    return pd.Series({
        "total_weight_g": total_weight,
        "total_volume_ml": total_volume
    })

In [ ]:

extracted = dff["catalog_content"].apply(extract_product_info).apply(pd.Series)


test_1 = pd.concat([df[["sample_id"]], extracted], axis=1)



test_1[["weight_g", "volume_ml"]]=test_1.apply(standardize_units, axis=1)
test_1[["total_weight_g", "total_volume_ml"]] = test_1.apply(total, axis=1)

test_1['total_qty'] = (
    test_1['pack_count'].fillna(1).astype(float).replace(0, 1) *
    test_1['quantity_per_pack'].fillna(1).astype(float).replace(0, 1)
)

In [ ]:
test_1.to_csv("cleaned_test_data.csv", index=False)